## Particle-Based Surface Shape Modeling
## PART 2 - Align and Scale Meshes

```
conda activate ECN_REDEV_SSM
```



In [9]:
# IMPORTS

import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from pathlib import Path
from scipy.ndimage import zoom
from scipy.spatial import procrustes, cKDTree, KDTree
import pyvista as pv
import vtk



### 1. Load Meshes

In [10]:
import glob
import pyvista as pv

# Directory containing your VTK files
output_dir = './OUTPUT/'

# Load vtk meshes
shape_iso_surfaces = []
for file in glob.glob(output_dir + '*.vtk'):
    shape_iso_surfaces.append(pv.read(file))

# Plot the two shapes side by side
plotter1 = pv.Plotter(shape=(1, 2), notebook=False)
plotter1.subplot(0, 0)
plotter1.add_mesh(shape_iso_surfaces[0], color='tan', show_edges=False)
plotter1.subplot(0, 1)
plotter1.add_mesh(shape_iso_surfaces[1], color='tan', show_edges=False)
plotter1.link_views()
plotter1.show()

# Plot the two shapes together
plotter2 = pv.Plotter(notebook=False)
plotter2.add_mesh(shape_iso_surfaces[0], color='red', show_edges=False)
plotter2.add_mesh(shape_iso_surfaces[1], color='blue', show_edges=False)
plotter2.show()



### 2. Align Meshes

To inspect how mutliple segmentation are spatially aligned with respect to each other, we will visualize their surfaces in the same rendering window. 

In [11]:
# CENTER AND SCALE MESHES

for mesh in shape_iso_surfaces:
    # Convert to vtkPolyData if necessary
    if not isinstance(mesh, vtk.vtkPolyData):
        mesh = mesh.extract_surface()
    
    # Get the points of the mesh
    points = mesh.points
    print(len(points))
    # Compute the centroid of the mesh
    centroid = np.mean(points, axis=0)
    # Center the mesh at the origin
    points -= centroid
    # Compute the scale factor (maximum distance from origin)
    distances = np.linalg.norm(points, axis=1)
    scale_factor = np.max(distances)
    # Scale the mesh to unit size
    points /= scale_factor
    mesh.points = points

# Plot the two shapes together
plotter2 = pv.Plotter(notebook=False)
plotter2.add_mesh(shape_iso_surfaces[0], color='red', show_edges=False)
plotter2.add_mesh(shape_iso_surfaces[1], color='blue', show_edges=False)
plotter2.show()


56994
45538


In [12]:
# METHOD 1: DISTANCE BASED OPTIMIZATION
"""
def vtk_mesh_to_pyvista(mesh):
    print("Converting VTK mesh to PyVista mesh...")
    if not isinstance(mesh, vtk.vtkPolyData):
        raise TypeError("Input mesh must be a vtkPolyData")
    pv_mesh = pv.wrap(mesh)
    return pv_mesh

# Function to sample random points on a mesh
def sample_random_points_on_mesh(mesh, n_points):
    print(f"Sampling {n_points} random points on the mesh...")
    mesh = mesh.triangulate()
    mesh_with_areas = mesh.compute_cell_sizes(length=False, area=True, volume=False)
    areas = mesh_with_areas['Area']
    total_area = np.sum(areas)
    probabilities = areas / total_area
    cumulative_areas = np.cumsum(probabilities)
    random_values = np.random.rand(n_points)
    triangle_indices = np.searchsorted(cumulative_areas, random_values)

    points = []
    for triangle_idx in triangle_indices:
        cell = mesh.faces[triangle_idx * 4:(triangle_idx + 1) * 4]
        indices = cell[1:4]
        triangle = mesh.points[indices]
        r1 = np.random.rand()
        r2 = np.random.rand()
        sqrt_r1 = np.sqrt(r1)
        barycentric_coords = [1 - sqrt_r1, sqrt_r1 * (1 - r2), sqrt_r1 * r2]
        point = (barycentric_coords[0] * triangle[0] +
                 barycentric_coords[1] * triangle[1] +
                 barycentric_coords[2] * triangle[2])
        points.append(point)
    return np.array(points)

# Function to optimize particle positions on a mesh
def particle_based_optimization(mesh, n_points=100, iterations=200, step_size=0.01):
    print(f"Starting particle-based optimization with {n_points} points and {iterations} iterations...")
    particles = sample_random_points_on_mesh(mesh, n_points)
    tree = cKDTree(mesh.points)
    for i in range(iterations):
        if i % 10 == 0:
            print(f"    Iteration {i}/{iterations}...")
        # Compute pairwise distances between particles
        particle_tree = cKDTree(particles)
        # Find pairs of particles that are too close
        pairs = particle_tree.query_pairs(r=0.05)
        # Compute forces and update particle positions
        forces = np.zeros_like(particles)
        for idx1, idx2 in pairs:
            vec = particles[idx1] - particles[idx2]
            dist = np.linalg.norm(vec)
            if dist > 1e-5:
                force = vec / dist**2
                forces[idx1] += force
                forces[idx2] -= force
        particles += step_size * forces
        distances, idx = tree.query(particles)
        particles = mesh.points[idx]
    print("Optimization complete.")
    return particles

# Function to plot particles and their indices
def plot_particles_with_indices(mesh, optimized_points, subplot_index, plotter):
    plotter.subplot(0, subplot_index)
    plotter.add_mesh(mesh, color='lightgray', opacity=0.5, show_edges=False)
    plotter.add_points(optimized_points, color='blue', point_size=10)

    # Adding particle indices as labels
    for idx, point in enumerate(optimized_points):
        plotter.add_point_labels(optimized_points[idx:idx+1], [str(idx)], point_size=0, font_size=10, text_color='red')

# Initialize list for storing PyVista meshes and optimized points
pv_meshes = []
optimized_points_list = []

# Convert VTK meshes to PyVista meshes and optimize particle positions
for sw_mesh in shape_iso_surfaces:
    if not isinstance(sw_mesh, vtk.vtkPolyData):
        sw_mesh = sw_mesh.extract_surface()
    pv_mesh = vtk_mesh_to_pyvista(sw_mesh)
    pv_meshes.append(pv_mesh)
    optimized_points = particle_based_optimization(pv_mesh, n_points=100)
    optimized_points_list.append(optimized_points)

# Plot the optimized points on both meshes side by side with labels
p = pv.Plotter(shape=(1, 2), notebook=False)
plot_particles_with_indices(pv_meshes[0], optimized_points_list[0], 0, p)
plot_particles_with_indices(pv_meshes[1], optimized_points_list[1], 1, p)

p.link_views()
p.show(title='Particle-Based Optimization on Meshes with Indices')

# Save the optimized points
np.save(output_dir + 'optimized_points_mesh1.npy', optimized_points_list[0])
np.save(output_dir + 'optimized_points_mesh2.npy', optimized_points_list[1])

print("Point distributions have been saved.")
"""
None

In [13]:
# METHOD 2 : CRITICAL POINTS BASED OPTIMIZATION
"""
# Directory to save output points
output_dir = "./output/"

# Function to convert VTK mesh to PyVista mesh
def vtk_mesh_to_pyvista(mesh):
    print("Converting VTK mesh to PyVista mesh...")
    if not isinstance(mesh, vtk.vtkPolyData):
        raise TypeError("Input mesh must be a vtk.vtkPolyData")
    pv_mesh = pv.wrap(mesh)
    return pv_mesh

# Identify critical zones in the mesh (e.g., based on curvature)
def identify_critical_zones(mesh):
    print("Identifying critical zones in the mesh based on curvature...")
    curvature = mesh.curvature(curv_type="mean")  # Using mean curvature as an indicator
    curvature_threshold = np.percentile(curvature, 20)  # Example threshold for critical zones
    critical_zones = curvature > curvature_threshold
    return critical_zones

# Particle optimization: move points towards critical zones
def optimize_particle_positions(mesh, critical_zones, num_particles=100):
    print("Optimizing particle positions...")
    # Get points from the mesh
    points = mesh.points
    
    # Create a mask for critical zones
    critical_points = points[critical_zones]
    
    # Initialize random particles across the mesh
    particle_indices = np.random.choice(range(len(points)), size=num_particles, replace=False)
    particles = points[particle_indices]
    
    # Optimize particle positions using proximity to critical zones
    tree = KDTree(critical_points)
    for i in range(5000):  # Optimization loop
        distances, indices = tree.query(particles)
        new_positions = critical_points[indices]
        particles = 0.9 * particles + 0.1 * new_positions  # Move particles towards critical zones
    
    return particles

# Main Process
pv_meshes = []
optimized_points_list = []

# Convert VTK meshes to PyVista meshes
for sw_mesh in shape_iso_surfaces:
    if not isinstance(sw_mesh, vtk.vtkPolyData):
        sw_mesh = sw_mesh.extract_surface()
    pv_mesh = vtk_mesh_to_pyvista(sw_mesh)
    pv_meshes.append(pv_mesh)

# Perform particle optimization across all meshes
for i, mesh in enumerate(pv_meshes):
    critical_zones = identify_critical_zones(mesh)
    if np.any(critical_zones):  # Check if there are any critical zones
        optimized_points = optimize_particle_positions(mesh, critical_zones)
        optimized_points_list.append(optimized_points)
    else:
        print(f"No critical zones found for mesh {i}, skipping optimization.")

# Plot optimized points for each mesh
p = pv.Plotter(shape=(1, len(pv_meshes)), notebook=False)
for i, mesh in enumerate(pv_meshes):
    p.subplot(0, i)
    p.add_mesh(mesh, color='lightgrey', opacity=0.5)
    if i < len(optimized_points_list):
        p.add_points(optimized_points_list[i], color='blue', point_size=5)

p.link_views()
p.show()

# Save the optimized points for each mesh
for i, optimized_points in enumerate(optimized_points_list):
    np.save(output_dir + f'optimized_points_mesh_{i+1}.npy', optimized_points)

print("Point distributions for all meshes have been saved.")

"""

None